# 0. Environment Setup

In [1]:
# ### Install spconv-plus

# !pip install pccm==0.3.4 ccimport==0.3.7
# import torch
# import shutil
# import os
# gpu = torch.cuda.get_device_name().replace(' ', '_').lower()
# cumm_whl = f'cumm/{gpu}/cumm_cu120-0.2.8-cp39-cp39-linux_x86_64.whl'
# spconv_plus_whl = f'spconv-plus/{gpu}/spconv_cu120-2.1.21-cp39-cp39-linux_x86_64.whl'

# os.environ['CUMM_CUDA_VERSION'] = "12.0"

# if not os.path.exists(cumm_whl):
#     if not os.path.exists('cumm'):
#         !git clone --recursive https://github.com/FindDefinition/cumm.git
#     os.environ['CUMM_DISABLE_JIT'] = "1"
#     %cd cumm
#     !git checkout v0.2.8
#     !python setup.py bdist_wheel
#     os.makedirs(gpu, exist_ok=True)
#     %cd ../
#     shutil.move(f'cumm/dist/cumm_cu120-0.2.8-cp39-cp39-linux_x86_64.whl', cumm_whl)
# !pip install $cumm_whl

# if not os.path.exists(spconv_plus_whl):
#     if not os.path.exists('spconv-plus'):
#         !git clone --recursive https://github.com/dvlab-research/spconv-plus.git
#     os.environ['SPCONV_DISABLE_JIT'] = "1"
#     %cd spconv-plus
#     !python setup.py bdist_wheel
#     os.makedirs(gpu, exist_ok=True)
#     %cd ../
#     shutil.move(f'spconv-plus/dist/spconv_cu120-2.1.21-cp39-cp39-linux_x86_64.whl', spconv_plus_whl)
# !pip install $spconv_plus_whl

# #### Install remaining dependencies
# !pip uninstall torch torchvision torchaudio --yes
# !pip install torch==2.2.0 torchvision==0.17.0 torchaudio==2.2.0 --index-url https://download.pytorch.org/whl/cu121
# !pip install lightning 
# !pip install torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html --upgrade
# !sudo apt-get install libsparsehash-dev
# !pip install timm
# !pip install wandb

# # #### KPConv kernel
# # %cd KPConv/cpp_wrappers
# # !sh compile_wrappers.sh
# # %cd ../../

# !pip install numba strictyaml
# !pip install dropblock

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 31.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 59.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 67.4 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.6.0-py2.py3-none-any.whl size=117030 sha256=e616472f4805610ae16e16f1d02f90edbcbaa0e5c1a20bd52e6cda3ce422cdef
  Stored in directory: /root/.cache/pip/wheels/ad/22/bc/87e9046fdfb73bf7217f0a3f10fbade8a4465f2cb0e6b21bf3
Successfully built fire
Processing ./cumm/nvidia_rtx_a6000/cumm_cu120-0.2.8-cp39-cp39-linux_x86_64.whl
Processing ./spconv-plus/nvidia_rtx_a6000/spconv_cu120-2.1.21-cp39-cp39-linux_x86_64.whl
Found existing installation: torch 1.12.1+cu116
Uninstalling torch-1.12.1+cu116:
  Successfully uninstal

In [1]:
import os
import cv2
import json
import logging
import shutil
import copy
import zipfile
import torch
import torchvision
from torch import nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import numpy as np
from tqdm import tqdm
import multiprocessing
from multiprocessing.pool import Pool
import functools
import itertools
import math
import glob

In [2]:
os.environ["SPCONV_DEBUG_SAVE_PATH"] = "/notebooks/spconv_log/error.log"

In [3]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [4]:
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'backend:cudaMallocAsync'
# os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [5]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [6]:
DATA_PATH = '/storage/SemanticKITTI/dataset'
SAVE_PATH = 'result'
PRED_PATH = 'predictions'

if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH, exist_ok=True)
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH, exist_ok=True)
os.makedirs(PRED_PATH, exist_ok=True)

In [7]:
# with zipfile.ZipFile('train_2.zip') as zf:
#     zf.extractall(os.path.join(DATA_PATH, 'sequences'))
# os.remove('train_2.zip')

In [8]:
import spconv.pytorch as spconv
from spconv.pytorch import SparseConvTensor
import lightning as L

# from torch_geometric.data import Batch

from model_spconv.layers import *
from model_spconv.networks import *
from model_spconv.dataloader import *
from model_spconv.utils import downsample_label, process_point_label

In [9]:
stage = 2
resume_training = False

# 1. Dataset preprocessing

In [10]:
data_config_path = 'model_spconv/cfgs/semantic-kitti.yaml'
load_path ='result/s2_checkpoints_caps_2d/SSC_last.ckpt'

In [11]:
# downsample_label(DATA_PATH, device=device)
# process_point_label(DATA_PATH, device=device)

In [12]:
train_dataset = SSCDataset(
    data_root=DATA_PATH,
    data_config_path=data_config_path,
    split="train",
    augmentation=True,
    stage=stage
)

valid_dataset = SSCDataset(
    data_root=DATA_PATH,
    data_config_path=data_config_path,
    split="valid",
    augmentation=False,
    stage=stage
)

test_dataset = SSCDataset(
    data_root=DATA_PATH,
    data_config_path=data_config_path,
    split="test",
    augmentation=False,
    stage=2
)

input 4649
point_input 4649
point_label 4649
label 4649
invalid 4649
occluded 4649
input 815
point_input 815
point_label 815
label 815
invalid 815
occluded 815
input 3901
point_input 3901


In [13]:
train_dataloader = DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
    drop_last=False,
    pin_memory=False,
    num_workers=8,
    collate_fn=custom_collate_fn
)

valid_dataloader = DataLoader(
    valid_dataset,
    batch_size=16,
    shuffle=False,
    drop_last=False,
    pin_memory=False,
    num_workers=8,
    collate_fn=custom_collate_fn
)

test_dataloader = DataLoader(
    test_dataset,
    batch_size=1,
    shuffle=False,
    drop_last=False,
    pin_memory=False,
    num_workers=8,
    collate_fn=custom_collate_fn
)

# 2. Training

In [14]:
# define model
if resume_training:
    model = SSCNetworkCaps(
        data_config_path=data_config_path
    )
else:
    model = SSCNetworkCaps.load_from_checkpoint(load_path)

model.set_train_stage(stage)

model.freeze_stage1_param()

model.set_optimizers(torch.optim.Adam(
    model.parameters(),
    lr=1e-3,
    betas=(0.9, 0.999),
    eps=1e-08,
    weight_decay=1e-3,
))

# model.set_schedulers({
#     "scheduler": torch.optim.lr_scheduler.ExponentialLR(
#         model.optim,
#         gamma = 0.98,
#     ),
#     "interval": "step",
#     "frequency": 60,
# })

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    model.optim,
    100,
)

# for i in range(77):
#     scheduler.step()

model.set_schedulers({
    "scheduler": scheduler,
    "interval": "epoch",
    "frequency": 1,
})

[IOU EVAL] IGNORE:  tensor([], dtype=torch.int64)
[IOU EVAL] INCLUDE:  tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
        18, 19])
[IOU EVAL] IGNORE:  tensor([0])
[IOU EVAL] INCLUDE:  tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
        19])


In [15]:
# torch.save(model.cfeat_extractor.state_dict(), 'cfeat.pt')
# torch.save(model.ss_net.state_dict(), 'ssnet.pt')

In [16]:
# model.cfeat_extractor.load_state_dict(torch.load('cfeat.pt'))
# model.ss_net.load_state_dict(torch.load('ssnet.pt'))

In [17]:
inv_remap_lut = SSCDataset.get_inv_remap_lut(valid_dataset.learning_map_inv)
model.to(device)
model.eval()
model.reset_time()
with torch.inference_mode():
    with torch.no_grad():
        for t, data_batch in tqdm(enumerate(valid_dataloader), total=len(valid_dataloader)):
            model(data_batch, save=True, save_path='val_predictions_1/sequences/', inv_remap_lut=inv_remap_lut, device=device)
avg_time = model.cal_avg_time()
avg_time

  0%|          | 0/51 [00:00<?, ?it/s]

0.10155614636427054

In [18]:
avg_time = model.cal_avg_time()
avg_time

0.05242509169081237

In [25]:
import shutil
shutil.make_archive('val_predictions_1', 'zip', 'val_predictions_1')

'/notebooks/val_predictions_1.zip'

In [21]:
torch.autograd.set_detect_anomaly(True)
torch.set_float32_matmul_precision('high')

In [22]:
#define callback for saving model checkpoint
from lightning.pytorch.callbacks import ModelCheckpoint

checkpoint_path = os.path.join(SAVE_PATH, 's2_checkpoints_caps_2d')
# checkpoint_path = 'result/s2_checkpoints_final'
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path, exist_ok=True)

# sem_optim_targets = [
#     # 'val_total_loss',
#     'iou_mean',
#     # 'train_total_loss',
# ]
    
optim_targets = [
    # 'val_total_loss',
    'iou_completion',
    'iou_mean',
    # 'train_total_loss',
]

checkpoint_callback = [
    ModelCheckpoint(
        monitor=None,
        dirpath=checkpoint_path,
        filename=f'SSC_last',
        every_n_epochs=1,
        save_on_train_epoch_end=True,
    )
]

for target in optim_targets:
    checkpoint_callback.append(ModelCheckpoint(
        monitor=target,
        dirpath=checkpoint_path,
        filename=f'SSC_sem-{{epoch:02d}}-{{{target}:.2f}}',
        save_top_k=3,
        save_last=False,
        mode='min' if 'loss' in target else 'max',
        every_n_epochs=2,
        save_on_train_epoch_end=False,
        # every_n_train_steps=250,
    ))

In [23]:
# define callback to reset experts score
# class ResetImportanceScore(L.Callback):
#     def on_train_epoch_end(self, trainer, pl_module):
#         pl_module.expert_scores.reset_score()
        
class EvalLogCallback(L.Callback):
    def on_validation_epoch_end(self, trainer, pl_module):
        pl_module.eval_metrics_log()
        
# reset_score_callback = ResetImportanceScore()
eval_log_callback = EvalLogCallback()

In [24]:
# import wandb
# wandb.util.generate_id()

In [25]:
#define logger
from lightning.pytorch.loggers import WandbLogger


log_path = os.path.join(SAVE_PATH, 'logs')
if not os.path.exists(log_path):
    os.makedirs(log_path, exist_ok=True)

#2qdv3jbn 2z30z0c7 1wky4xit xgn3mx88 294dlh1a 158mq3ce 2gchciys
wandb_logger = WandbLogger(
    project="SemanticSceneCompletion",
    log_model=True,
    save_dir=log_path,
    name='SSCv5_final_stage2_caps_2d',
    entity='ssc_project',
    id="2gchciys",
    resume=True,
)

In [ ]:
#define trainer and train model
trainer = L.Trainer(
    max_epochs=120,
    callbacks=checkpoint_callback +  [eval_log_callback],
    logger=wandb_logger,
    log_every_n_steps=50,
    check_val_every_n_epoch=2,
    accumulate_grad_batches=1
)
last_checkpoint = 'result/s2_checkpoints_caps_2d/SSC_last.ckpt'
if not os.path.exists(last_checkpoint):
    last_checkpoint = None
# last_checkpoint=None
trainer.fit(model, train_dataloader, valid_dataloader,ckpt_path=last_checkpoint)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
wandb: Currently logged in as: hontrn9122 (ssc_project). Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.9/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /notebooks/result/s2_checkpoints_caps_2d exists and is not empty.
Restoring states from the checkpoint path at result/s2_checkpoints_caps_2d/SSC_last.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type            | Params
----------------------------------------------------
0 | cfeat_extractor | cylinder_fea    | 116 K 
1 | ss_net          | Asymm_3d_spconv | 55.8 M
2 | pc_refinement   | PCRefinement    | 574 K 
3 | sc_net          | CapsBEV_UNet    | 41.8 M
----------------------------------------------------
42.4 M    Trainable params
55.9 M    Non-trainable params
98.3 M    Total params
393.135   Total estimated model params size (MB)
Restored all states from the checkpoint at result/s2_checkpoints_caps_2d/SSC_last.ckpt


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [ ]:
print('ok')